In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import numpy as np
import random
import pandas as pd
from copy import deepcopy as dp

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold

from sklearn.model_selection import KFold

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.modules.loss import _WeightedLoss
from torchvision import transforms
import torchvision.models as models

import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau

from pickle import dump, load

import gc

## NAIVE MODEL

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed) 
    tf.random.set_seed(seed)

seed_everything(42)

In [ ]:
input_dir = '../input/lish-moa/'
train_features = pd.read_csv(input_dir+'train_features.csv')
train_targets = pd.read_csv(input_dir+'train_targets_scored.csv')
test_features = pd.read_csv(input_dir+'test_features.csv')
train_drug = pd.read_csv(input_dir+'train_drug.csv')

In [ ]:
# Convert categorical variable into dummy/indicator variables
train_features = pd.get_dummies(train_features, columns=['cp_type', 'cp_dose'])
test_features = pd.get_dummies(test_features, columns=['cp_type', 'cp_dose'])

# Remove sig_id
all_dfs = (train_features, train_targets, test_features)

for df in all_dfs:
    if 'sig_id' in df.columns:
        df.drop('sig_id', axis=1, inplace=True)

In [ ]:
GENE_COLS = [col for col in train_features.columns if col.startswith('g-')]
CELL_COLS = [col for col in train_features.columns if col.startswith('c-')]

train_features_gene = train_features.loc[:, GENE_COLS]
test_features_gene = test_features.loc[:, GENE_COLS]
train_features_cell = train_features.loc[:, CELL_COLS]
test_features_cell = test_features.loc[:, CELL_COLS]

N_COMP_GENE = 90
N_COMP_CELL = 50

# PCA for gene
pca_gene_train = PCA(n_components=N_COMP_GENE).fit_transform(train_features_gene)
pca_gene_train = pd.DataFrame(data=pca_gene_train, columns=[f'pc-g-{i}' for i in range(N_COMP_GENE)])
pca_gene_test = PCA(n_components=N_COMP_GENE).fit_transform(test_features_gene)
pca_gene_test = pd.DataFrame(data=pca_gene_test, columns=[f'pc-g-{i}' for i in range(N_COMP_GENE)])

# PCA for cell
pca_cell_train = PCA(n_components=N_COMP_CELL).fit_transform(train_features_cell)
pca_cell_train = pd.DataFrame(data=pca_cell_train, columns=[f'pc-c-{i}' for i in range(N_COMP_CELL)])
pca_cell_test = PCA(n_components=N_COMP_CELL).fit_transform(test_features_cell)
pca_cell_test = pd.DataFrame(data=pca_cell_test, columns=[f'pc-c-{i}' for i in range(N_COMP_CELL)])

In [ ]:
# Appending new components to existing features
pca_train = train_features.copy()
# pca_train = pd.concat([pca_train, pca_gene_train, pca_cell_train], axis=1)

pca_test = test_features.copy()
# pca_test = pd.concat([pca_test, pca_gene_test, pca_cell_test], axis=1)

In [ ]:
import matplotlib.pyplot as plt
pca = PCA().fit(train_features_cell)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In [ ]:
### MODIFY THIS FOR DIFFERENT DATASETS ###

THE_TRAIN = pca_train
THE_TEST = pca_test

##########################################

pred_val = np.zeros((THE_TRAIN.shape[0], 206))
pred_test = np.zeros((THE_TEST.shape[0], 206))

features = THE_TRAIN.values
targets = train_targets.values
tests = THE_TEST.values

validation_scores = []

In [ ]:
!nvidia-smi

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
def get_model(train_df, *units):
    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.Input(shape=(train_df.shape[1],)))
    model.add(tf.keras.layers.BatchNormalization())
    
    for u in units[:-1]:
        model.add(tf.keras.layers.Dense(units=u, activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dropout(0.5))
    
    model.add(tf.keras.layers.Dense(units=units[-1], activation="sigmoid"))
    
    model.compile(loss='binary_crossentropy', optimizer='adam')
    
    return model

def run(features, targets, tests, pred, pe, n_split=5):
    kfoldnumber = 0

    for train_index, validation_index in KFold(n_split).split(features):
        kfoldnumber += 1
        print(f'{"#" * 30} Fold number {kfoldnumber} {"#" * 30}')

        reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss',
                                           factor=0.1, patience=3, verbose=0,
                                           epsilon=1e-4, mode='min')
        
        nn_layers = (512, 1024, 206)
        model = get_model(features, *nn_layers)
        #model.summary()

        model.fit(features[train_index],
                  targets[train_index],
                  batch_size=128,
                  epochs=35,
                  validation_data=(features[validation_index], targets[validation_index]),
                  verbose=0,
                  callbacks=[reduce_lr_loss])

        print()
        print('train loss:\t', model.evaluate(features[train_index], targets[train_index],
                                     verbose=0, batch_size=128))
        
        validate_score = model.evaluate(features[validation_index],
                                         targets[validation_index],
                                        verbose=0, batch_size=128)
        print('validate loss:\t', validate_score)
        validation_scores.append(validate_score)
        

        print()
        print('predict validation...')

        pred[validation_index] = model.predict(features[validation_index],
                                              verbose=0, batch_size=128)

        print('predict test...')

        pe += model.predict(tests, verbose=0, batch_size=128) / n_split
        print()
        
        del model    
        gc.collect()
        torch.cuda.empty_cache()
    
    print('###########################################################################\n\nFIN')
    
    return pred, pe

In [ ]:
!nvidia-smi

In [ ]:
# Run model
pred_val, pred_test = run(features, targets, tests, pred_val, pred_test)

# Average prediction validation score
print(sum(validation_scores)/(len(validation_scores)))

In [ ]:
!nvidia-smi

In [ ]:
columns = pd.read_csv(input_dir + "train_targets_scored.csv")
columns.drop('sig_id', axis=1, inplace=True)
sub1 = pd.DataFrame(data=pred_test, columns=columns.columns)
sample = pd.read_csv(input_dir + "sample_submission.csv")
sub1.insert(0, column='sig_id', value=sample['sig_id'])

sub1.to_csv('submission.csv', index=False)
print(sub1)

## DEEP INSIGHT + RESNET


In [ ]:
# # TabNet
# !pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl pytorch-tabnet

# # Tabnet 
# from torch.optim.lr_scheduler import ReduceLROnPlateau
# from pytorch_tabnet.metrics import Metric
# from pytorch_tabnet.tab_model import TabNetRegressor

# feature transformation - fit
def norm_fit(df_1,saveM = True, sc_name = 'zsco'):   
    from sklearn.preprocessing import StandardScaler,MinMaxScaler,MaxAbsScaler,RobustScaler,Normalizer,QuantileTransformer,PowerTransformer
    ss_1_dic = {'zsco':StandardScaler(),
                'mima':MinMaxScaler(),
                'maxb':MaxAbsScaler(), 
                'robu':RobustScaler(),
                'norm':Normalizer(), 
                'quan':QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal"),
                'powe':PowerTransformer()}
    ss_1 = ss_1_dic[sc_name]
    df_2 = pd.DataFrame(ss_1.fit_transform(df_1),index = df_1.index,columns = df_1.columns)
    if saveM == False:
        return(df_2)
    else:
        return(df_2,ss_1)

# feature transformation - trans
def norm_tra(df_1,ss_x):
    df_2 = pd.DataFrame(ss_x.transform(df_1),index = df_1.index,columns = df_1.columns)
    return(df_2)

# frequency 
def f_table(list1):
    table_dic = {}
    for i in list1:
        if i not in table_dic.keys():
            table_dic[i] = 1
        else:
            table_dic[i] += 1
    return(table_dic)

# seed for reproducibility
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
import warnings
warnings.filterwarnings('ignore')

seed_everything(seed=42)

# input data dir
input_dir = '../input/lish-moa/'
# upload model dataset from training kernel outputs
mod_path1 = '../input/deepinsight-resnet-training-final/deepinsight-resnet-training/'          # Resnet

In [ ]:
cache_dir = "/root/.cache/torch/hub/checkpoints/"
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

!cp ../input/resnet50/* $cache_dir

!ls $cache_dir

model = models.resnet50(pretrained = False)

In [ ]:
SEED = [0]
input_dir = '../input/lish-moa/'

sc_dic = {}
feat_dic = {}
train_features = pd.read_csv(input_dir+'train_features.csv')
train_targets_scored = pd.read_csv(input_dir+'train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(input_dir+'train_targets_nonscored.csv')
test_features = pd.read_csv(input_dir+'test_features.csv')
sample_submission = pd.read_csv(input_dir+'sample_submission.csv')
train_drug = pd.read_csv(input_dir+'train_drug.csv')

target_cols = train_targets_scored.drop('sig_id', axis=1).columns.values.tolist()
target_nonsc_cols = train_targets_nonscored.drop('sig_id', axis=1).columns.values.tolist()

######## non-score targets highly correlated with scored targets will be used in pretrain ########
nonctr_id = train_features.loc[:,'sig_id'].tolist()
tmp_con1 = [i in nonctr_id for i in train_targets_scored['sig_id']]
mat_cor = pd.DataFrame(np.corrcoef(train_targets_scored.drop('sig_id',axis = 1)[tmp_con1].T,
                      train_targets_nonscored.drop('sig_id',axis = 1)[tmp_con1].T))
mat_cor2 = mat_cor.iloc[(train_targets_scored.shape[1]-1):,0:train_targets_scored.shape[1]-1]
mat_cor2.index = target_nonsc_cols
mat_cor2.columns = target_cols
mat_cor2 = mat_cor2.dropna()
mat_cor2_max = mat_cor2.abs().max(axis = 1)

q_n_cut = 0.9
target_nonsc_cols2 = mat_cor2_max[mat_cor2_max > np.quantile(mat_cor2_max,q_n_cut)].index.tolist()
print(len(target_nonsc_cols2))

GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]
feat_dic['gene'] = GENES
feat_dic['cell'] = CELLS
 
train = train_features.merge(train_targets_scored, on='sig_id')
train = train.merge(train_targets_nonscored[['sig_id']+target_nonsc_cols2], on='sig_id')

target = train[['sig_id']+target_cols]
target_ns = train[['sig_id']+target_nonsc_cols2]

train0 = train
test = test_features

for df in [train0, test]:
    df['cp_type'] = df['cp_type'].map({'ctl_vehicle': 0, 'trt_cp': 1})
    df['cp_dose'] = df['cp_dose'].map({'D1': 0, 'D2': 1})
    df['cp_time'] = df['cp_time'].map({24: 0, 48: 0.5, 72: 1})

target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

# drug ids
tar_sig = target['sig_id'].tolist()
train_drug = train_drug.loc[[i in tar_sig for i in train_drug['sig_id']]]
target = target.merge(train_drug, on='sig_id', how='left') 

# LOCATE DRUGS
vc = train_drug.drug_id.value_counts()
vc1 = vc.loc[vc <= 19].index
vc2 = vc.loc[vc > 19].index

feature_cols = []
for key_i in feat_dic.keys():
    value_i = feat_dic[key_i]
    print(key_i,len(value_i))
    feature_cols += value_i
len(feature_cols)
feature_cols0 = dp(feature_cols)
    
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

In [ ]:
for seed in SEED:
    seed_everything(seed=seed)
    folds = train0.copy()
    feature_cols = dp(feature_cols0)
    
    # Kfold - leave drug out
    target2 = target.copy()
    dct1 = {}; dct2 = {}
    skf = MultilabelStratifiedKFold(n_splits = 5) # , shuffle = True, random_state = seed
    tmp = target2.groupby('drug_id')[target_cols].mean().loc[vc1]
    tmp_idx = tmp.index.tolist()
    tmp_idx.sort()
    tmp_idx2 = random.sample(tmp_idx,len(tmp_idx))
    tmp = tmp.loc[tmp_idx2]
    for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[target_cols])):
        dd = {k:fold for k in tmp.index[idxV].values}
        dct1.update(dd)

    skf = MultilabelStratifiedKFold(n_splits = 5) # , shuffle = True, random_state = seed
    tmp = target2.loc[target2.drug_id.isin(vc2)].reset_index(drop = True)
    tmp_idx = tmp.index.tolist()
    tmp_idx.sort()
    tmp_idx2 = random.sample(tmp_idx,len(tmp_idx))
    tmp = tmp.loc[tmp_idx2]
    for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[target_cols])):
        dd = {k:fold for k in tmp.sig_id[idxV].values}
        dct2.update(dd)

    target2['kfold'] = target2.drug_id.map(dct1)
    target2.loc[target2.kfold.isna(),'kfold'] = target2.loc[target2.kfold.isna(),'sig_id'].map(dct2)
    target2.kfold = target2.kfold.astype(int)

    folds['kfold'] = target2['kfold'].copy()

    train = folds.copy()
    test_ = test.copy()

    # HyperParameters
    DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
    EPOCHS = 20
    BATCH_SIZE = 128
    LEARNING_RATE = 1e-3
    WEIGHT_DECAY = 1e-5
    NFOLDS = 5
    EARLY_STOPPING_STEPS = 10
    EARLY_STOP = False

    n_comp1 = 50
    n_comp2 = 15

    num_features=len(feature_cols) + n_comp1 + n_comp2
    num_targets=len(target_cols)
    num_targets_0=len(target_nonsc_cols2)
    hidden_size=4096

    tar_freq = np.array([np.min(list(f_table(train[target_cols].iloc[:,i]).values())) for i in range(len(target_cols))])
    tar_weight0 = np.array([np.log(i+100) for i in tar_freq])
    tar_weight0_min = dp(np.min(tar_weight0))
    tar_weight = tar_weight0_min/tar_weight0
    np.mean(tar_weight)
    pos_weight = torch.tensor(tar_weight).to(DEVICE)
    
    class SmoothBCEwLogits(_WeightedLoss):
        def __init__(self, weight=None, reduction='mean', smoothing=0.0):
            super().__init__(weight=weight, reduction=reduction)
            self.smoothing = smoothing
            self.weight = weight
            self.reduction = reduction

        @staticmethod
        def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
            assert 0 <= smoothing < 1
            with torch.no_grad():
                targets = targets * (1.0 - smoothing) + 0.5 * smoothing
            return targets

        def forward(self, inputs, targets):
            targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
                self.smoothing)
            loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight,
                                                      pos_weight = pos_weight)

            if  self.reduction == 'sum':
                loss = loss.sum()
            elif  self.reduction == 'mean':
                loss = loss.mean()

            return loss
        
    # Alicia - DeepInsight
    class LogScaler:
        """Log normalize and scale data

        Log normalization and scaling procedure as described as norm-2 in the
        DeepInsight paper supplementary information.

        Note: The dimensions of input matrix is (N samples, d features)
        """
        def __init__(self):
            self._min0 = None
            self._max = None

        """
        Use this as a preprocessing step in inference mode.
        """
        def fit(self, X, y=None):
            # Min. of training set per feature
            self._min0 = X.min(axis=0)

            # Log normalized X by log(X + _min0 + 1)
            X_norm = np.log(X + np.repeat(np.abs(self._min0)[np.newaxis, :], X.shape[0], axis=0) + 1).clip(min=0, max=None)
            # Global max. of training set from X_norm
            self._max = X_norm.max()

        """
        For training set only.
        """
        def fit_transform(self, X, y=None):
            # Min. of training set per feature
            self._min0 = X.min(axis=0)

            # Log normalized X by log(X + _min0 + 1)
            X_norm = np.log(X + np.repeat(np.abs(self._min0)[np.newaxis, :], X.shape[0], axis=0) + 1).clip(min=0, max=None)

            # Global max. of training set from X_norm
            self._max = X_norm.max()

            # Normalized again by global max. of training set
            return (X_norm / self._max).clip(0, 1)

        """
        For validation and test set only.
        """
        def transform(self, X, y=None): 
            # Adjust min. of each feature of X by _min0
            for i in range(X.shape[1]):
                X[:, i] = X[:, i].clip(min=self._min0[i], max=None)

            # Log normalized X by log(X + _min0 + 1)
            X_norm = np.log(
                X +
                np.repeat(np.abs(self._min0)[np.newaxis, :], X.shape[0], axis=0) +
                1).clip(min=0, max=None)

            # Normalized again by global max. of training set
            return (X_norm / self._max).clip(0, 1)
    
    class DeepInsightTransformer:
        """Transform features to an image matrix using dimensionality reduction

        This class takes in data normalized between 0 and 1 and converts it to a
        CNN compatible 'image' matrix

        """
        def __init__(self,
                     feature_extractor='tsne',
                     perplexity=30,
                     pixels=100,
                     random_state=None,
                     n_jobs=None):
            """Generate an ImageTransformer instance

            Args:
                feature_extractor: string of value ('tsne', 'pca', 'kpca') or a
                    class instance with method `fit_transform` that returns a
                    2-dimensional array of extracted features.
                pixels: int (square matrix) or tuple of ints (height, width) that
                    defines the size of the image matrix.
                random_state: int or RandomState. Determines the random number
                    generator, if present, of a string defined feature_extractor.
                n_jobs: The number of parallel jobs to run for a string defined
                    feature_extractor.
            """
            self.random_state = random_state
            self.n_jobs = n_jobs

            if isinstance(feature_extractor, str):
                fe = feature_extractor.casefold()
                if fe == 'tsne_exact'.casefold():
                    fe = TSNE(n_components=2,
                              metric='cosine',
                              perplexity=perplexity,
                              n_iter=1000,
                              method='exact',
                              random_state=self.random_state,
                              n_jobs=self.n_jobs)
                elif fe == 'tsne'.casefold():
                    fe = TSNE(n_components=2,
                              metric='cosine',
                              perplexity=perplexity,
                              n_iter=1000,
                              method='barnes_hut',
                              random_state=self.random_state,
                              n_jobs=self.n_jobs)
                elif fe == 'pca'.casefold():
                    fe = PCA(n_components=2, random_state=self.random_state)
                elif fe == 'kpca'.casefold():
                    fe = KernelPCA(n_components=2,
                                   kernel='rbf',
                                   random_state=self.random_state,
                                   n_jobs=self.n_jobs)
                else:
                    raise ValueError(("Feature extraction method '{}' not accepted"
                                      ).format(feature_extractor))
                self._fe = fe
            elif hasattr(feature_extractor, 'fit_transform') and \
                    inspect.ismethod(feature_extractor.fit_transform):
                self._fe = feature_extractor
            else:
                raise TypeError('Parameter feature_extractor is not a '
                                'string nor has method "fit_transform"')

            if isinstance(pixels, int):
                pixels = (pixels, pixels)

            # The resolution of transformed image
            self._pixels = pixels
            self._xrot = None

        def fit(self, X, y=None, plot=False):
            """Train the image transformer from the training set (X)

            Args:
                X: {array-like, sparse matrix} of shape (n_samples, n_features)
                y: Ignored. Present for continuity with scikit-learn
                plot: boolean of whether to produce a scatter plot showing the
                    feature reduction, hull points, and minimum bounding rectangle

            Returns:
                self: object
            """
            # Transpose to get (n_features, n_samples)
            X = X.T

            # Perform dimensionality reduction
            x_new = self._fe.fit_transform(X)

            # Get the convex hull for the points
            chvertices = ConvexHull(x_new).vertices
            hull_points = x_new[chvertices]

            # Determine the minimum bounding rectangle
            mbr, mbr_rot = self._minimum_bounding_rectangle(hull_points)

            # Rotate the matrix
            # Save the rotated matrix in case user wants to change the pixel size
            self._xrot = np.dot(mbr_rot, x_new.T).T

            # Determine feature coordinates based on pixel dimension
            self._calculate_coords()

            # plot rotation diagram if requested
            if plot is True:
                # Create subplots
                fig, ax = plt.subplots(1, 1, figsize=(10, 7), squeeze=False)
                ax[0, 0].scatter(x_new[:, 0],
                                 x_new[:, 1],
                                 cmap=plt.cm.get_cmap("jet", 10),
                                 marker="x",
                                 alpha=1.0)
                ax[0, 0].fill(x_new[chvertices, 0],
                              x_new[chvertices, 1],
                              edgecolor='r',
                              fill=False)
                ax[0, 0].fill(mbr[:, 0], mbr[:, 1], edgecolor='g', fill=False)
                plt.gca().set_aspect('equal', adjustable='box')
                plt.show()
            return self

        @property
        def pixels(self):
            """The image matrix dimensions

            Returns:
                tuple: the image matrix dimensions (height, width)

            """
            return self._pixels

        @pixels.setter
        def pixels(self, pixels):
            """Set the image matrix dimension

            Args:
                pixels: int or tuple with the dimensions (height, width)
                of the image matrix

            """
            if isinstance(pixels, int):
                pixels = (pixels, pixels)
            self._pixels = pixels
            # recalculate coordinates if already fit
            if hasattr(self, '_coords'):
                self._calculate_coords()

        def _calculate_coords(self):
            """Calculate the matrix coordinates of each feature based on the
            pixel dimensions.
            """
            ax0_coord = np.digitize(self._xrot[:, 0],
                                    bins=np.linspace(min(self._xrot[:, 0]),
                                                     max(self._xrot[:, 0]),
                                                     self._pixels[0])) - 1
            ax1_coord = np.digitize(self._xrot[:, 1],
                                    bins=np.linspace(min(self._xrot[:, 1]),
                                                     max(self._xrot[:, 1]),
                                                     self._pixels[1])) - 1
            self._coords = np.stack((ax0_coord, ax1_coord))

        def transform(self, X, empty_value=0):
            """Transform the input matrix into image matrices

            Args:
                X: {array-like, sparse matrix} of shape (n_samples, n_features)
                    where n_features matches the training set.
                empty_value: numeric value to fill elements where no features are
                    mapped. Default = 0 (although it was 1 in the paper).

            Returns:
                A list of n_samples numpy matrices of dimensions set by
                the pixel parameter
            """
            # Group by location (x1, y1) of each feature
            # Tranpose to get (n_features, n_samples)
            img_coords = pd.DataFrame(np.vstack(
                (self._coords, X.clip(0, 1))).T).groupby(
                    [0, 1],  # (x1, y1)
                    as_index=False).mean()

            img_matrices = []
            blank_mat = np.zeros(self._pixels)
            if empty_value != 0:
                blank_mat[:] = empty_value
            for z in range(2, img_coords.shape[1]):
                img_matrix = blank_mat.copy()
                img_matrix[img_coords[0].astype(int),
                           img_coords[1].astype(int)] = img_coords[z]
                img_matrices.append(img_matrix)
                
            img_matrices = np.array([self._mat_to_rgb(m) for m in img_matrices])
                
            return img_matrices
        
        def transform_3d(self, X, empty_value=0):
            """Transform the input matrix into image matrices

            Args:
                X: {array-like, sparse matrix} of shape (n_samples, n_features)
                    where n_features matches the training set.
                empty_value: numeric value to fill elements where no features are
                    mapped. Default = 0 (although it was 1 in the paper).

            Returns:
                A list of n_samples numpy matrices of dimensions set by
                the pixel parameter
            """

            # Group by location (x1, y1) of each feature
            # Tranpose to get (n_features, n_samples)
            img_coords = pd.DataFrame(np.vstack(
                (self._coords, X.clip(0, 1))).T).groupby(
                    [0, 1],  # (x1, y1)
                    as_index=False)
            avg_img_coords = img_coords.mean()
            min_img_coords = img_coords.min()
            max_img_coords = img_coords.max()

            img_matrices = []
            blank_mat = np.zeros((3, self._pixels[0], self._pixels[1]))
            if empty_value != 0:
                blank_mat[:, :, :] = empty_value
            for z in range(2, avg_img_coords.shape[1]):
                img_matrix = blank_mat.copy()
                img_matrix[0, avg_img_coords[0].astype(int),
                           avg_img_coords[1].astype(int)] = avg_img_coords[z]
                img_matrix[1, min_img_coords[0].astype(int),
                           min_img_coords[1].astype(int)] = min_img_coords[z]
                img_matrix[2, max_img_coords[0].astype(int),
                           max_img_coords[1].astype(int)] = max_img_coords[z]
                img_matrices.append(img_matrix)

            return img_matrices

        def fit_transform(self, X, empty_value=0):
            """Train the image transformer from the training set (X) and return
            the transformed data.

            Args:
                X: {array-like, sparse matrix} of shape (n_samples, n_features)
                empty_value: numeric value to fill elements where no features are
                    mapped. Default = 0 (although it was 1 in the paper).

            Returns:
                A list of n_samples numpy matrices of dimensions set by
                the pixel parameter
            """
            self.fit(X)
            return self.transform(X, empty_value=empty_value)
        
        def fit_transform_3d(self, X, empty_value=0):
            """Train the image transformer from the training set (X) and return
            the transformed data.

            Args:
                X: {array-like, sparse matrix} of shape (n_samples, n_features)
                empty_value: numeric value to fill elements where no features are
                    mapped. Default = 0 (although it was 1 in the paper).

            Returns:
                A list of n_samples numpy matrices of dimensions set by
                the pixel parameter
            """
            self.fit(X)
            return self.transform_3d(X, empty_value=empty_value)

        def feature_density_matrix(self):
            """Generate image matrix with feature counts per pixel

            Returns:
                img_matrix (ndarray): matrix with feature counts per pixel
            """
            fdmat = np.zeros(self._pixels)
            # Group by location (x1, y1) of each feature
            # Tranpose to get (n_features, n_samples)
            coord_cnt = (
                pd.DataFrame(self._coords.T).assign(count=1).groupby(
                    [0, 1],  # (x1, y1)
                    as_index=False).count())
            fdmat[coord_cnt[0].astype(int),
                  coord_cnt[1].astype(int)] = coord_cnt['count']
            return fdmat

        @staticmethod
        def _minimum_bounding_rectangle(hull_points):
            """Find the smallest bounding rectangle for a set of points.

            Modified from JesseBuesking at https://stackoverflow.com/a/33619018
            Returns a set of points representing the corners of the bounding box.

            Args:
                hull_points : an nx2 matrix of hull coordinates

            Returns:
                (tuple): tuple containing
                    coords (ndarray): coordinates of the corners of the rectangle
                    rotmat (ndarray): rotation matrix to align edges of rectangle
                        to x and y
            """

            pi2 = np.pi / 2.

            # Calculate edge angles
            edges = hull_points[1:] - hull_points[:-1]
            angles = np.arctan2(edges[:, 1], edges[:, 0])
            angles = np.abs(np.mod(angles, pi2))
            angles = np.unique(angles)

            # Find rotation matrices
            rotations = np.vstack([
                np.cos(angles),
                np.cos(angles - pi2),
                np.cos(angles + pi2),
                np.cos(angles)
            ]).T
            rotations = rotations.reshape((-1, 2, 2))

            # Apply rotations to the hull
            rot_points = np.dot(rotations, hull_points.T)

            # Find the bounding points
            min_x = np.nanmin(rot_points[:, 0], axis=1)
            max_x = np.nanmax(rot_points[:, 0], axis=1)
            min_y = np.nanmin(rot_points[:, 1], axis=1)
            max_y = np.nanmax(rot_points[:, 1], axis=1)

            # Find the box with the best area
            areas = (max_x - min_x) * (max_y - min_y)
            best_idx = np.argmin(areas)

            # Return the best box
            x1 = max_x[best_idx]
            x2 = min_x[best_idx]
            y1 = max_y[best_idx]
            y2 = min_y[best_idx]
            rotmat = rotations[best_idx]

            # Generate coordinates
            coords = np.zeros((4, 2))
            coords[0] = np.dot([x1, y2], rotmat)
            coords[1] = np.dot([x2, y2], rotmat)
            coords[2] = np.dot([x2, y1], rotmat)
            coords[3] = np.dot([x1, y1], rotmat)

            return coords, rotmat
        
        @staticmethod
        def _mat_to_rgb(mat):
            """Convert image matrix to numpy rgb format
            Args:
                mat: {array-like} (M, N)
            Returns:
                An numpy.ndarry (M, N, 3) with orignal values repeated across
                RGB channels.
            """
            return np.repeat(mat[:, :, np.newaxis], 3, axis=2)

    class TrainDataset:
#         def __init__(self, features, targets):
#             self.features = features
#             self.targets = targets

#         def __len__(self):
#             return (self.features.shape[0])

#         def __getitem__(self, idx):
#             dct = {
#                 'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
#                 'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
#             }
#             return dct
        
        ####### Non-swap ######
        def __init__(self, features, labels, transformer):
            self.features = features
            self.labels = labels
            self.transformer = transformer

        def __getitem__(self, index):
            normalized = self.features[index, :]
            normalized = np.expand_dims(normalized, axis=0)

            # Note: we are setting empty_value=1 to follow the setup in the paper
            image = self.transformer.transform(normalized, empty_value=1)[0]
            
            preprocess = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])

            return {"x": preprocess(image).type(torch.float), "y": torch.tensor(self.labels[index, :],dtype=torch.float)}
        
        
        def __len__(self):
            return self.features.shape[0]

    class TestDataset:
#         def __init__(self, features):
#             self.features = features

#         def __len__(self):
#             return (self.features.shape[0])

#         def __getitem__(self, idx):
#             dct = {
#                 'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
#             }
#             return dct

        def __init__(self, features, transformer):
            self.features = features
            self.transformer = transformer

        def __getitem__(self, index):
            normalized = self.features[index, :]
            normalized = np.expand_dims(normalized, axis=0)

            # Note: we are setting empty_value=1 to follow the setup in the paper
            image = self.transformer.transform(normalized, empty_value=1)[0]
            
            preprocess = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            
            return  {"x": preprocess(image).type(torch.float)}

        def __len__(self):
            return self.features.shape[0]

    def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
        model.train()
        final_loss = 0

        for data in dataloader:
            optimizer.zero_grad()
            inputs, targets = data['x'].to(device), data['y'].to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()
            scheduler.step()

            final_loss += loss.item()

        final_loss /= len(dataloader)

        return final_loss


    def valid_fn(model, loss_fn, dataloader, device):
        model.eval()
        final_loss = 0
        valid_preds = []

        for data in dataloader:
            inputs, targets = data['x'].to(device), data['y'].to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)

            final_loss += loss.item()
            valid_preds.append(outputs.sigmoid().detach().cpu().numpy())

        final_loss /= len(dataloader)
        valid_preds = np.concatenate(valid_preds)

        return final_loss, valid_preds

    def inference_fn(model, dataloader, device):
        model.eval()
        preds = []

        for data in dataloader:
            inputs = data['x'].to(device)
            with torch.no_grad():
                outputs = model(inputs)

            preds.append(outputs.sigmoid().detach().cpu().numpy())

        preds = np.concatenate(preds)

        return preds


    def run_training(fold, seed):

        seed_everything(seed)

        trn_idx = train[train['kfold'] != fold].index
        val_idx = train[train['kfold'] == fold].index

        train_df = train[train['kfold'] != fold].reset_index(drop=True).copy()
        valid_df = train[train['kfold'] == fold].reset_index(drop=True).copy()

        x_train, y_train,y_train_ns = train_df[feature_cols].values, train_df[target_cols].values,train_df[target_nonsc_cols2].values
        x_valid, y_valid,y_valid_ns  =  valid_df[feature_cols].values, valid_df[target_cols].values,valid_df[target_nonsc_cols2].values
        x_test = test_[feature_cols].values
        
        def load_pickle(model_output_folder, seed_i, fold_i, name):
            return load(open(f"{model_output_folder}/seed{seed_i}_fold{fold_i}_{name}.pkl", 'rb'))
        
        # LogScaler (Norm-2 Normalization)
#         print("Running norm-2 normalization ......")
        
        scaler = load_pickle(mod_path1 + "Resnet_deepinsight", seed, fold, "log-scaler")
        x_test = scaler.transform(x_test)
        x_train = scaler.transform(x_train)
        x_valid = scaler.transform(x_valid)
        
        # Load DeepInsight Feature Map
        transformer = load_pickle(mod_path1 + "Resnet_deepinsight", seed, fold,
                                  "deepinsight-transform")
        
        
        train_dataset = TrainDataset(x_train, y_train, transformer)
        valid_dataset = TrainDataset(x_valid, y_valid, transformer)
        
#         train_dataset = TrainDataset(x_train, y_train)
#         valid_dataset = TrainDataset(x_valid, y_valid)
        trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                                  max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))

        loss_tr = SmoothBCEwLogits(smoothing = 0.001)
        loss_va = nn.BCEWithLogitsLoss()    

        early_stopping_steps = EARLY_STOPPING_STEPS
        early_step = 0

        oof = np.zeros((len(train), len(target_cols)))
        best_loss = np.inf

        mod_name = mod_path1 + f"FOLD_mod11_{seed}_{fold}_.pth"
        
        model.fc = nn.Sequential(nn.Linear(2048, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, num_targets))
        
        model.load_state_dict(torch.load(mod_name))
        model.to(DEVICE)
        
        oof[val_idx] = inference_fn(model, validloader, DEVICE)
        
        #--------------------- PREDICTION---------------------
        testdataset = TestDataset(x_test, transformer)
        testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)

        predictions = np.zeros((len(test_), len(target_cols)))
        predictions = inference_fn(model, testloader, DEVICE)
        return oof, predictions

    def run_k_fold(NFOLDS, seed):
        oof = np.zeros((len(train), len(target_cols)))
        predictions = np.zeros((len(test), len(target_cols)))

        for fold in range(NFOLDS):
            print('seed:',seed,'fold:',fold)
            oof_, pred_ = run_training(fold, seed)

            predictions += pred_ / NFOLDS
            oof += oof_

        return oof, predictions

    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)
    
    oof_tmp = dp(oof)
    oof_tmp = oof_tmp * len(SEED) / (SEED.index(seed)+1)
    sc_dic[seed] = np.mean([log_loss(train[target_cols].iloc[:,i],oof_tmp[:,i]) for i in range(len(target_cols))])

print(np.mean([log_loss(train[target_cols].iloc[:,i],oof[:,i]) for i in range(len(target_cols))]))

train0[target_cols] = oof
test[target_cols] = predictions

sub2 = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)

### mod1 ###
train0_1 = train0.copy()
sub_1 = sub2.copy()

pd.DataFrame(sc_dic,index=['sc']).T

In [ ]:
# final submission
sub2.to_csv('submission.csv', index=False)

In [ ]:
submission = pd.read_csv(input_dir+'sample_submission.csv')
submission.iloc[:, 1:] = 0
submission.iloc[:, 1:] = sub1.iloc[:,1:]*0.9 + sub2.iloc[:,1:]*0.1 
submission.to_csv('submission.csv', index=False)